In [1]:
import pandas as pd

In [143]:
def read_mosnet_results(filepath, dataset_folder, dataset_name):
    vgg19_results_style_id = []
    vgg19_results_content_id = []
    vgg19_results_mos_score = []
    DATASET = dataset_folder

    with open(filepath, 'r') as f:
        for line in f:
            if 'Average' in line or 'outputs' in line:
                continue
            metrics = line.split(DATASET)[1]
#             print(metrics)
            filename, mos_score = metrics.strip().split(' ')
            if '_' in filename:
                filename = filename[:filename.index('_')]
#             print(filename)
            content_id, style_id = filename.split('x')
            if '.wav' in style_id:
                style_id = style_id[:style_id.index('.')]
            vgg19_results_content_id.append(content_id)
            vgg19_results_style_id.append(style_id)
            vgg19_results_mos_score.append(float(mos_score))
    df = pd.DataFrame()
    df['content_id'] = pd.Series(vgg19_results_content_id)
    df['style_id'] = pd.Series(vgg19_results_style_id)
    df[dataset_name+'_mos_score'] = pd.Series(vgg19_results_mos_score)
    return df

In [144]:
vgg_df = read_mosnet_results('./vgg_data/MOSnet_result_raw.txt', 'vgg_data/', 'vgg')

In [145]:
autovc_df = read_mosnet_results('./results/MOSnet_result_raw.txt', 'results/', 'autovc')

In [146]:
shallow_cnn_df = read_mosnet_results('./neural-style-audio-tf/MOSnet_result_raw.txt', 'neural-style-audio-tf/', 'ulyanov')

In [147]:
def get_aggregated_metadata(vgg_df):
    metadata = pd.read_csv('test_vox1_metadata.csv')
    
    metadata['content_id'] = metadata['VoxCeleb1 ID']
    metadata = metadata.drop(columns='VoxCeleb1 ID')
    merged_df = pd.merge(vgg_df, metadata, on='content_id')
    metadata['style_id'] = metadata['content_id']
    merged_df['converted_style_id'] = merged_df['style_id']
    df = pd.merge(merged_df, metadata[['style_id','Gender', 'Nationality']], on='style_id', suffixes=('_original', '_converted'))
    return df

In [148]:
metadata = pd.read_csv('test_vox1_metadata.csv')

In [149]:
metadata['content_id'] = metadata['VoxCeleb1 ID']

In [150]:
metadata = metadata.drop(columns='VoxCeleb1 ID')

In [151]:
merged_df = pd.merge(vgg_df, metadata, on='content_id')


In [152]:
metadata['style_id'] = metadata['content_id']

In [153]:
merged_df['converted_style_id'] = merged_df['style_id']

In [154]:
merged_vgg_df = get_aggregated_metadata(vgg_df)

In [155]:
merged_shallow_cnn_df = get_aggregated_metadata(shallow_cnn_df)

In [156]:
merged_autovc_df = get_aggregated_metadata(autovc_df)

In [157]:
merged_vgg_df.groupby(['Gender_original', 'Gender_converted']).mean()

vgg_mos_score  Unnamed: 0
Gender_original Gender_converted                           
f               f                      2.377000  269.000000
                m                      2.498000  269.000000
m               f                      2.427750  271.000000
                m                      2.417652  271.130435

In [ ]:
merged_vgg_df.groupby(['Nationality_original', 'Nationality_converted']).mean()

In [158]:
merged_shallow_cnn_df.groupby(['Gender_original', 'Gender_converted']).mean()

ulyanov_mos_score  Unnamed: 0
Gender_original Gender_converted                               
f               f                          2.364500  270.666667
                m                          2.438909  269.000000
m               f                          2.648745  273.765957
                m                          2.691048  272.333333

In [161]:
merged_shallow_cnn_df.groupby(['Nationality_original', 'Nationality_converted']).mean()

ulyanov_mos_score  Unnamed: 0
Nationality_original Nationality_converted                               
Ireland              USA                             2.544000  279.000000
Spain                Ireland                         2.520000  276.000000
                     UK                              2.469000  276.000000
                     USA                             2.543000  276.000000
UK                   Canada                          2.556000  270.000000
                     France                          2.691000  270.000000
                     India                           2.755000  270.000000
                     Ireland                         2.787600  271.400000
                     Mexico                          2.848000  270.000000
                     Norway                          2.770000  270.000000
                     Spain                           2.369333  272.333333
                     UK                              2.707571  271.285714
                     USA                             2.623829  271.142857
USA                  Canada                          2.707000  278.000000
                     Ireland                         2.643667  273.666667
                     Spain                           2.347000  271.750000
                     UK                              2.614333  272.250000
                     USA                             2.693947  273.342105

In [160]:
merged_autovc_df.groupby(['Gender_original', 'Gender_converted']).mean()

autovc_mos_score  Unnamed: 0
Gender_original Gender_converted                              
f               f                         3.081118  270.235294
                m                         3.220923  269.384615
m               f                         2.770459  271.351351
                m                         3.032298  271.052632

In [125]:
metadata = pd.read_csv('test_vox1_metadata.csv')

metadata['content_id'] = metadata['VoxCeleb1 ID']
metadata = metadata.drop(columns='VoxCeleb1 ID')
pd.merge(autovc_df, metadata, on='content_id')
metadata['style_id'] = metadata['content_id']
merged_df['converted_style_id'] = merged_df['style_id']
merged_autovc_df = pd.merge(merged_df, metadata[['style_id','Gender', 'Nationality']], on='style_id', suffixes=('_original', '_converted'))


In [162]:
len(merged_autovc_df) + len(merged_shallow_cnn_df) + len(merged_vgg_df)

310